## notebookExample for nPnB-QUICK-BEC
        March 2025, 
        Bruno Gaume <bruno.gaume@iscpif.fr>
## Reference:
    nPnB framework Reference:
        Two antagonistic objectives for one multi-scale graph clustering framework,
        Bruno Gaume, Ixandra Achitouv, David Chavalarias Nature Scientific Reports (2025)
        https://www.nature.com/articles/s41598-025-90454-w

    BEC2 Algorithm Reference:
        BEC.2: A fast and relevant Multi-Scale Graph Clustering algorithm in nPnB framework,
        Bruno Gaume (2025)

## Before clic on [Kernel]-->[Restart Kernel and Run All Cells ...]
    we have to compile nPnB-QUICK-BEC_C++/nPnB-QUICK-BEC.cpp (see ReadMe.txt)

    


In [ ]:
# =============================================================================
# Import
# =============================================================================
import os
import sys
import math
# -----------------------------------------------------------------------------
beep = lambda x: os.system("echo -n '\a';sleep 0.1;" * x)
here=os.path.abspath(os.getcwd())+"/"
print("here ='%s'"%here)
# =============================================================================
import nPnB_QUICK_BEC_Interface as nPnB
PathnPnB=here
# ===================================================================

sys.path
devnull = open(os.devnull, 'w')
sys.stderr=devnull
junk=beep(1) # everything is well loaded

In [ ]:

# =============================================================================
# Some functions
# =============================================================================
def beta2s(beta):
    """
    """
    return (2*(math.atan(beta)))/math.pi

def s2beta(s):
    """
    """
    return (math.tan((math.pi*s)/2))
    
def PrintClust(g,c):
    """
    """
    c.sort(key = lambda z : len(z), reverse=True)
    for m in c:
        m.sort(reverse=False)
        
    member=[[] for _ in range(g["nbv"])]
    for i in range(len(c)):
        m=c[i]
        for j in range(len(m)):
            x = m[j]
            member[x].append(i)

    # modules
    CH="Modules:\n"
    for i in range(len(c)):
        m=c[i]
        CH=CH+"  %i (|m.%i|=%i): {"%(i, i, len(m))
        for j in range(len(m)):
            x = m[j]
            CH=CH+str(x)
            if j<len(m)-1:
                CH=CH+", "
        CH=CH+"}"
        if i<len(c)-1:
            CH=CH+"\n"
    print(CH)

    # nodes in overlaps
    CH=""
    for i in range(g["nbv"]):
        x=member[i]
        if len(x) > 1:
            CH=CH+"  node %i in: "%(i)
            for j in range(len(x)):
                CH=CH+"m.%i"%(x[j])
                if j<len(x)-1:
                    CH=CH+", "
            if i<g["nbv"]-1:
                CH=CH+"\n"
    if not (CH == ""):
        CH="Nodes in overlaps:\n"+CH
        print(CH)
    print("")

# ACTION

In [ ]:

# WITH: Zachary's karate club"

namegraph = "karateClub";
# https://www.jstor.org/stable/3629752
# https://en.wikipedia.org/wiki/Zachary%27s_karate_club
# https://www.pnas.org/doi/10.1073/pnas.122653799

# THE TYPE OF GRAPH
directed=False

# THE NUMBER OF NODES
nbv=34

# THE WEIGHTED EDGES (Node 0 stands for the instructor, node 33 for the club administrator/president)
#w=1
w=1.618034
edges = [((0, 1), w), ((0, 2), w), ((0, 3), w), ((0, 4), w), ((0, 5), w), ((0, 6), w), ((0, 7), w), ((0, 8), w), ((0, 10), w),
         ((0, 11), w), ((0, 12), w), ((0, 13), w), ((0, 17), w), ((0, 19), w), ((0, 21), w), ((0, 31), w), ((1, 2), w), ((1, 3), w),
         ((1, 7), w), ((1, 13), w), ((1, 17), w), ((1, 19), w), ((1, 21), w), ((1, 30), w), ((2, 3), w), ((2, 7), w), ((2, 27), w),
         ((2, 28), w), ((2, 32), w), ((2, 9), w), ((2, 8), w), ((2, 13), w), ((3, 7), w), ((3, 12), w), ((3, 13), w), ((4, 6), w),
         ((4, 10), w), ((5, 6), w), ((5, 10), w), ((5, 16), w), ((6, 16), w), ((8, 30), w), ((8, 32), w), ((8, 33), w), ((9, 33), w),
         ((13, 33), w), ((14, 32), w), ((14, 33), w), ((15, 32), w), ((15, 33), w), ((18, 32), w), ((18, 33), w), ((19, 33), w),
         ((20, 32), w), ((20, 33), w), ((22, 32), w), ((22, 33), w), ((23, 25), w), ((23, 27), w), ((23, 32), w), ((23, 33), w),
         ((23, 29), w), ((24, 25), w), ((24, 27), w), ((24, 31), w), ((25, 31), w), ((26, 29), w), ((26, 33), w), ((27, 33), w),
         ((28, 31), w), ((28, 33), w), ((29, 32), w), ((29, 33), w), ((30, 32), w), ((30, 33), w), ((31, 32), w), ((31, 33), w),
         ((32, 33), w)]

# nPnB-QUICK-BEC can work on graphs with multiple directed or undirected edges.
# For example, [... ((4,7), 2), ((4,7), 0.5), ((7,4), 10), ...] is equivalent to [... ((4,7), 12.5), ...]
# And is counted as a single undirected edge {4,7} of weight 12.5. 

# THE GRAPH
graph={"directed":directed,
   "nbv":nbv,
   "edges":edges}

SmallGraph = (graph["nbv"] <= 100)

# A GROUND TRUTH (see https://www.pnas.org/doi/10.1073/pnas.122653799)
namegroundtruth= "AfterSplit";
groundtruth = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 16, 17, 19, 21],
              [9, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]]


In [ ]:
# =============================================================================
# EXAMPLE: how to call nPnB_mmm & nPnB_overlaps

# nPnB-QUICK-BEC(G, sp, so) is a kind of telescope for observing the graph G modeling a complex system 
# where nodes represent the basic lowest-level entities and edges represent their interactions.

# sp defines the desired scale of description of the modules as highest-level entities
# and so the level of overlap of these entities.
# =============================================================================

# INPUTS @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# sp and so in [0,1] and Epsilon in [0,1[

# The smaller sp, the finer the scale of description, producing many small, dense modules.
# The larger sp, the coarser the scale of description, producing fewer, larger, less dense modules.

# The smaller Epsilon, the more we favor quality.
# The larger Epsilon, the more we favor computation speed (at the expense of quality).
# Epsilon=0.01 is a good compromise between speed and quality.

sp=0.81; Epsilon=0.01; so=0.81; 
# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

print("=============================================================================");
print("%s: |V|=%i"%(namegraph, graph["nbv"]));
print("=============================================================================\n");

#=================================================
# INTRINSIC QUALITY OF AFTERSPLIT
#=================================================
print("-------------------------------------------------------------------");
print("C: %s"%(namegroundtruth))
print("-------------------------------------------------------------------");
print("|C|=%i"%(len(groundtruth)))
if SmallGraph:
    PrintClust(graph, groundtruth);

print("Intrinsic Quality (against %s):"%(namegraph));
(TP, TN, FP, FN) = nPnB.TPTNFPFN_Intrinsic(PathnPnB,  graph, groundtruth)
(P, R, F_1) = nPnB.PRF(s2beta(0.5), TP, TN, FP, FN)
print("  [TP=%.2f, TN=%.2f, FP=%.2f, FN=%.2f]"%(TP, TN, FP, FN));
print("  P=%.2f, R=%.2f, F_(s=0.50)=%.2f"%(P, R, F_1));
print("-------------------------------------------------------------------\n");
#=================================================
# PARTITIONAL CLUSTERING
#=================================================
print("-------------------------------------------------------------------");
print("Cp: nPnB-QUICK-BEC Partition of %s with sp=%.2f, Epsilon=%.2f:"%(namegraph, sp, Epsilon));
print("-------------------------------------------------------------------");
Cp, Pp_i, Rp_i, Fp_1_i, Fp_sp_i, TPp_i, TNp_i, FPp_i, FNp_i, Ctimep = nPnB.nPnB_mmm(PathnPnB, graph, sp=sp,
                                                                C0string=False, RandNode=False, Epsilon=Epsilon, Verbose=True);

print("|Cp|=%i (%.2f seconds)"%(len(Cp), Ctimep))
if SmallGraph:
    PrintClust(graph,Cp);

print("Intrinsic Quality (against %s):"%(namegraph));
(TPp_i, TNp_i, FPp_i, FNp_i) = nPnB.TPTNFPFN_Intrinsic(PathnPnB,  graph, Cp)
(Pp_i, Rp_i, Fp_1_i) = nPnB.PRF(s2beta(0.5), TPp_i, TNp_i, FPp_i, FNp_i)
(Pp_i, Rp_i, Fp_sp_i) = nPnB.PRF(s2beta(sp), TPp_i, TNp_i, FPp_i, FNp_i)
print("  [TP=%.2f, TN=%.2f, FP=%.2f, FN=%.2f]"%(TPp_i, TNp_i, FPp_i, FNp_i));
print("  P=%.2f, R=%.2f, F_(s=0.50)=%.2f, F_(s=sp=%.2f)=%.2f\n"%(Pp_i, Rp_i, Fp_1_i, sp, Fp_sp_i));

print("Extrinsic Quality (against %s):" %(namegroundtruth));
(TPp_e, TNp_e, FPp_e, FNp_e) = nPnB.TPTNFPFN_GOLD_C(PathnPnB, groundtruth, Cp, graph["nbv"])
(Pp_e, Rp_e, Fp_1_e)  = nPnB.PRF(s2beta(0.5), TPp_e, TNp_e, FPp_e, FNp_e)
(Pp_e, Rp_e, Fp_sp_e) = nPnB.PRF(s2beta(sp), TPp_e, TNp_e, FPp_e, FNp_e)
print("  [TP=%.2f, TN=%.2f, FP=%.2f, FN=%.2f]"%(TPp_e, TNp_e, FPp_e, FNp_e));
print("  P=%.2f, R=%.2f, F_(s=0.50)=%.2f, F_(s=sp=%.2f)=%.2f"%(Pp_e, Rp_e, Fp_1_e, sp, Fp_sp_e));
print("-------------------------------------------------------------------\n");

#=================================================
# CLUSTERING ALLOWING OVERLAPS
#=================================================
print("-------------------------------------------------------------------");
print("Co: nPnB-QUICK-BEC Overlaps from Cp with so=%.2f:"%(so));
print("-------------------------------------------------------------------");
C0string=nPnB.clust2string(Cp);
Co, Po_i, Ro_i, Fo_1_i, Fo_so_i, TPo_i, TNo_i, FPo_i, FNo_i, Ctimeo = nPnB.nPnB_overlaps(PathnPnB, graph, so=so,
                                                                C0string=C0string, RandNode=False, Verbose=True);

print("|Co|=%i (%.2f seconds)"%(len(Co), Ctimeo))
if SmallGraph:
    PrintClust(graph,Co);

print("Intrinsic Quality (against %s):"%(namegraph));
(TPo_i, TNo_i, FPo_i, FNo_i) = nPnB.TPTNFPFN_Intrinsic(PathnPnB, graph, Co)
(Po_i, Ro_i, Fo_1_i) = nPnB.PRF(s2beta(0.5), TPo_i, TNo_i, FPo_i, FNo_i);
(Po_i, Ro_i, Fo_sp_i) = nPnB.PRF(s2beta(sp), TPo_i, TNo_i, FPo_i, FNo_i);
print("  [TP=%.2f, TN=%.2f, FP=%.2f, FN=%.2f]"%(TPo_i, TNo_i, FPo_i, FNo_i));
print("  P=%.2f, R=%.2f, F_(s=0.50)=%.2f, F_(s=sp=%.2f)=%.2f\n"%(Po_i, Ro_i, Fo_1_i, sp, Fo_sp_i));

print("Extrinsic Quality (against %s):" %(namegroundtruth));
(TPo_e, TNo_e, FPo_e, FNo_e) = nPnB.TPTNFPFN_GOLD_C(PathnPnB, groundtruth, Co, graph["nbv"])
(Po_e, Ro_e, Fo_1_e)  = nPnB.PRF(s2beta(0.5), TPo_e, TNo_e, FPo_e, FNo_e)
(Po_e, Ro_e, Fo_sp_e) = nPnB.PRF(s2beta(sp), TPo_e, TNo_e, FPo_e, FNo_e)
print("  [TP=%.2f, TN=%.2f, FP=%.2f, FN=%.2f]"%(TPo_e, TNo_e, FPo_e, FNo_e));
print("  P=%.2f, R=%.2f, F_(s=0.50)=%.2f, F_(s=sp=%.2f)=%.2f"%(Po_e, Ro_e, Fo_1_e, sp, Fo_sp_e));
print("-------------------------------------------------------------------\n");
